In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
print(tf.__version__)

2.1.0


In [2]:
source_data = pd.read_csv('./气候数据集/DailyDelhiClimateTrain.csv')

In [14]:
# 生成1-1000的tf数组（可迭代对象）
range_ds = tf.data.Dataset.range(1002)
# 通过batch形式获取数据,drop_remainder 是剪掉尾巴
batchs = range_ds.batch(5,drop_remainder=True)
# take(4)代表只取五组数据
for i in batchs.take(5):
    print(i.numpy())

[0 1 2 3 4]
[5 6 7 8 9]
[10 11 12 13 14]
[15 16 17 18 19]
[20 21 22 23 24]


### 当需要一步一步获取数据和标签的时候

In [16]:
def dense_1_step(batch):
  # Shift features and labels one step relative to each other.
  # 切分batch数组以指定的形式返回
    return batch[:-1], batch[-1]

predict_dense_1_step = batchs.map(dense_1_step)

for features, label in predict_dense_1_step.take(3):
    print(features.numpy(), " => ", label.numpy())

[0 1 2 3]  =>  4
[5 6 7 8]  =>  9
[10 11 12 13]  =>  14
